In [3]:
import pandas as pd
import os
from datetime import datetime, timedelta

In [4]:
BOOKS_FILE = 'books.csv'
USERS_FILE = 'users.csv'
BORROW_FILE = 'borrowed.csv'

In [5]:
def load_data(file):
    return pd.read_csv(file) if os.path.exists(file) else pd.DataFrame()

def save_data(df, file):
    df.to_csv(file, index=False)

def signup():
    df = load_data(USERS_FILE)
    if df.empty or 'username' not in df.columns:
        df = pd.DataFrame(columns=["user_id", "username"])
    username = input("Enter new username: ")
    if username in df['username'].values:
        print("Username already exists.")
        return
    user_id = f"U{len(df) + 1}"
    new_user = pd.DataFrame([[user_id, username]], columns=["user_id", "username"])
    df = pd.concat([df, new_user], ignore_index=True)
    save_data(df, USERS_FILE)
    print("Signup successful. Please login.")

In [6]:
def login():
    df = load_data(USERS_FILE)
    if df.empty or 'username' not in df.columns:
        print("No users registered yet. Please sign up.")
        return
    username = input("Enter your username: ")
    user = df[df['username'] == username]
    if not user.empty:
        print(f"Welcome {username}!")
        main_menu(user.iloc[0]['user_id'])
    else:
        print("Invalid username. Try again.")



In [7]:
def add_book():
    df = load_data(BOOKS_FILE)
    if df.empty or 'title' not in df.columns:
        df = pd.DataFrame(columns=["book_id", "title", "author", "available"])
    book_id = f"B{len(df) + 1}"
    title = input("Enter book title: ")
    author = input("Enter author: ")
    new_book = pd.DataFrame([[book_id, title, author, 'yes']], columns=["book_id", "title", "author", "available"])
    df = pd.concat([df, new_book], ignore_index=True)
    save_data(df, BOOKS_FILE)
    print("Book added successfully.")




In [8]:
def borrow_book(user_id):
    books = load_data(BOOKS_FILE)
    if books.empty or 'available' not in books.columns:
        print("No books available.")
        return

    borrows = load_data(BORROW_FILE)
    if borrows.empty or 'book_id' not in borrows.columns:
        borrows = pd.DataFrame(columns=["user_id", "book_id", "borrow_date", "due_date"])

    book_id = input("Enter Book ID to borrow: ")
    if not books[(books['book_id'] == book_id) & (books['available'] == 'yes')].empty:
        books.loc[books['book_id'] == book_id, 'available'] = 'no'
        borrow_date = datetime.now()
        due_date = borrow_date + timedelta(days=7)
        new_borrow = pd.DataFrame([[user_id, book_id, borrow_date.strftime('%Y-%m-%d'), due_date.strftime('%Y-%m-%d')]],
                                  columns=["user_id", "book_id", "borrow_date", "due_date"])
        borrows = pd.concat([borrows, new_borrow], ignore_index=True)
        save_data(books, BOOKS_FILE)
        save_data(borrows, BORROW_FILE)
        print(f"Book borrowed. Return by {due_date.strftime('%Y-%m-%d')}")
    else:
        print("Book not available or doesn't exist.")


In [9]:
def return_book(user_id):
    books = load_data(BOOKS_FILE)
    borrows = load_data(BORROW_FILE)

    if books.empty or 'available' not in books.columns or borrows.empty or 'book_id' not in borrows.columns:
        print("No data found.")
        return

    book_id = input("Enter Book ID to return: ")
    if not borrows[(borrows['user_id'] == user_id) & (borrows['book_id'] == book_id)].empty:
        books.loc[books['book_id'] == book_id, 'available'] = 'yes'
        borrows = borrows[~((borrows['user_id'] == user_id) & (borrows['book_id'] == book_id))]
        save_data(books, BOOKS_FILE)
        save_data(borrows, BORROW_FILE)
        print("Book returned.")
    else:
        print("No record found for this book.")




In [10]:
def view_all_books():
    df = load_data(BOOKS_FILE)
    print(df.to_string(index=False))



In [11]:
def search_book():
    df = load_data(BOOKS_FILE)
    keyword = input("Enter title/author keyword: ").lower()
    results = df[df['title'].str.lower().str.contains(keyword) | df['author'].str.lower().str.contains(keyword)]
    print(results.to_string(index=False))



In [12]:
def view_overdue_books():
    borrows = load_data(BORROW_FILE)
    books = load_data(BOOKS_FILE)
    today = datetime.now().date()
    borrows['due_date'] = pd.to_datetime(borrows['due_date']).dt.date
    overdue = borrows[borrows['due_date'] < today]
    if overdue.empty:
        print("No overdue books.")
    else:
        merged = overdue.merge(books, on='book_id')
        print(merged[['book_id', 'title', 'author', 'due_date']].to_string(index=False))



In [13]:
def main_menu(user_id):
    while True:
        print("\n===== MAIN MENU =====")
        print("1. Add Book")
        print("2. Borrow Book")
        print("3. Return Book")
        print("4. View All Books")
        print("5. Search Book")
        print("6. View Overdue Books")
        print("7. Logout to Login Page")
        choice = input("Choose an option: ")
        if choice == '1':
            add_book()
        elif choice == '2':
            borrow_book(user_id)
        elif choice == '3':
            return_book(user_id)
        elif choice == '4':
            view_all_books()
        elif choice == '5':
            search_book()
        elif choice == '6':
            view_overdue_books()
        elif choice == '7':
            break
        else:
            print("Invalid choice.")



In [14]:
def user_menu():
    while True:
        print("\n-- Welcome to the Library System ---")
        print("1. Login")
        print("2. Signup")
        print("3. Exit")
        choice = input("Choose an option: ")
        if choice == '1':
            login()
        elif choice == '2':
            signup()
        elif choice == '3':
            print("Goodbye!")
            break
        else:
            print("Invalid input.")



In [15]:
if __name__ == "__main__":
    user_menu()



-- Welcome to the Library System ---
1. Login
2. Signup
3. Exit
Choose an option: 1
Enter your username: Admin
Welcome Admin!

===== MAIN MENU =====
1. Add Book
2. Borrow Book
3. Return Book
4. View All Books
5. Search Book
6. View Overdue Books
7. Logout to Login Page
Choose an option: 2
Enter Book ID to borrow: B1
Book borrowed. Return by 2025-08-15

===== MAIN MENU =====
1. Add Book
2. Borrow Book
3. Return Book
4. View All Books
5. Search Book
6. View Overdue Books
7. Logout to Login Page
Choose an option: 7

-- Welcome to the Library System ---
1. Login
2. Signup
3. Exit
Choose an option: 3
Goodbye!
